In [22]:
%reset
%load_ext autoreload
%autoreload 2

import sys

sys.path.append("../")
import torch
import numpy as np
import defmod as dm
import multimodule_usefulfunctions as mm 
import matplotlib.pyplot as plt

torch.set_default_tensor_type(torch.DoubleTensor)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
nb_pts1 = 10
nb_pts2 = 7
dim = 2

o1 = torch.tensor([1.,1.])
o2 = torch.tensor([5.,5.])
origin = [o1,o2]
radius = [2.,1.]
numberPoints = [nb_pts1, nb_pts2]
gd_list = mm.multipleCircles(origin, radius, numberPoints)

In [24]:

#nb_pts1 = 10
#gd_list = mm.multipleCircles([o1], [radius[1]], [nb_pts1])

In [25]:
# Defining the deformation modules
sigma1 = 10
manifold1 = dm.manifold.Landmarks(dim, nb_pts1, gd=gd_list[0].view(-1))
trans1 = dm.deformationmodules.Translations(manifold1, sigma1)

sigma2 = 10
manifold2 = dm.manifold.Landmarks(dim, nb_pts2, gd=gd_list[1].view(-1))
trans2 = dm.deformationmodules.Translations(manifold2, sigma2)

comp1 = dm.deformationmodules.CompoundModule([trans1])
comp2 = dm.deformationmodules.CompoundModule([trans2])

module_list = [comp1, comp2]
sigma_bg = 1
modules = dm.multishape.MultiShapeModule(module_list, sigma_bg)


In [26]:
# 2 Modules
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
cotan = [p1, p2, [p1, p2]]
#cotan = [torch.rand(nb_pts1*dim, requires_grad=True), -torch.rand(nb_pts2*dim, requires_grad=True), [torch.rand((nb_pts1)*dim, requires_grad=True),-torch.rand((nb_pts2)*dim, requires_grad=True)]]

modules.manifold.fill_cotan(cotan)



In [27]:
p = torch.cat([torch.ones_like(modules.manifold.unroll_cotan()[0]), -torch.ones_like(modules.manifold.unroll_cotan()[1])],0)

In [28]:
modules.manifold.fill_cotan(torch.cat([p,p],0))

print(torch.cat([p,p],0))
print(modules.manifold.cotan)

tensor([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1., -1., -1., -1., -1., -1., -1., -1., -1.,
        -1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1., -1.,
        -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.])
[[tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], requires_grad=True)], [tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       requires_grad=True)], [tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.], requires_grad=True), tensor([-1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.],
       requires_grad=True)]]


In [ ]:
# 1 Module
modules = dm.multishape.MultiShapeModule([trans1], sigma_bg)
# Setting cotangent variables
p1 = torch.cat([torch.ones(nb_pts1,1, requires_grad=True), torch.zeros(nb_pts1,1)],1).view(-1)
p2 = torch.cat([-torch.ones(nb_pts2,1,requires_grad=True), torch.zeros(nb_pts2,1)],1).view(-1)
#cotan2 = [p1, p2, [p1, p2]]
#cotan = [torch.rand(nb_pts1*dim, requires_grad=True), -torch.rand(nb_pts2*dim, requires_grad=True), [torch.rand((nb_pts1)*dim, requires_grad=True),-torch.rand((nb_pts2)*dim, requires_grad=True)]]

p1 = torch.ones(nb_pts1,dim, requires_grad=True).view(-1)
cotan = [p1, [p1]]
modules.manifold.fill_cotan(cotan)



In [ ]:
modules.background.fill_controls([torch.ones(modules.background.controls[i].shape) for i in range(2)])

modules.background.controls
modules.background.fill_controls_zero()
modules.background.controls[0].shape

In [8]:
constr_Id = dm.constraints.Identity()
#l, h = modules.compute_geodesic_variables(modules.manifold, constr_Id)

In [9]:
import defmod.hamiltonian_multishape as hamiltonian
H = hamiltonian.Hamiltonian_multi(modules, constr_Id)

In [16]:
# 2 Modules
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1,1),controls_list[1].view(-1,1), controls_list[-1][0].view(-1,1),controls_list[-1][1].view(-1,1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], controls[dim*nb_pts[0]:dim*(nb_pts[0]+nb_pts[1])], [controls[dim*n:dim*(n+nb_pts[0])], controls[dim*(n+nb_pts[0]):]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)

nb_pts = [nb_pts1, nb_pts2]

In [ ]:
# 1 Module
def controlslist2tensor(controls_list):
    return torch.cat([controls_list[0].view(-1,1), controls_list[-1][0].view(-1,1)])

def controlstensor2list(controls, nb_pts, dim):
    n = sum(nb_pts)
    return [controls[:dim*nb_pts[0]], [controls[dim*n:dim*(n+nb_pts[0])]]]

def gdtensor2list(gdtensor, nb_pts, dim):
    gdlist = []
    gdlist_bg = []
    j = 0
    n = sum(nb_pts)
    for i in range(len(nb_pts)):
        gdlist.append(gdtensor[dim*j:dim*(j+nb_pts[i])])
        gdlist_bg.append(gdtensor[dim*(n+j):dim*(n+j+nb_pts[i])])
        j = j + nb_pts[i]
    return [*gdlist, gdlist_bg]

def gdlist2tensor(gdlist):
    return torch.cat([*gdlist[:-1],*gdlist[-1]],0)

nb_pts = [nb_pts1]

In [ ]:
def test_gradcheck_list2tensor():
    def list2tensor(controls):
        
        controls = controlstensor2list(controls, nb_pts, dim)
        
        return controlslist2tensor(controls)
    
    controls = torch.cat([torch.ones(modules.controls[0].shape, requires_grad=True), torch.ones(modules.controls[0].shape, requires_grad=True)])

    return torch.autograd.gradcheck(list2tensor, controls, raise_exception=True)

test_gradcheck_list2tensor()

In [ ]:
def test_gradcheck_man_fill():
    def man_fill(gd):
        
        out = dm.manifold.Landmarks(2, 10)
        out.fill(man, copy=True)
        
        return out.gd
    
    return torch.autograd.gradcheck(man_fill, (gd), raise_exception=True)


man = dm.manifold.Landmarks(2, 10)
man.fill_gd(torch.rand(2*10, requires_grad=True))

gd = man.gd

test_gradcheck_man_fill()


In [ ]:
def test_gradcheck_background():
    def background_no_copy(gd):
        
        man = mod.manifold
        man2 = man

        return man2.gd
    
    def background_copy(gd):
        
        man=mod.manifold
        man2 = man.copy()

        return man2.gd
    
    def man_fill(gd):
        man = mod.manifold
        out = dm.manifold.CompoundManifold([trans1.manifold])
        out.fill(man, copy=True)
        
        return man.gd
    
    #return torch.autograd.gradcheck(background_no_copy, (gd), raise_exception=True)
    #return torch.autograd.gradcheck(background_copy, (gd), raise_exception=True)
    return torch.autograd.gradcheck(man_fill, (gd), raise_exception=True)

mod = trans1
gd = mod.manifold.gd

mod = dm.deformationmodules.Background([trans1], 1)
gd = mod.manifold.gd

#multimodule = dm.multishape.MultiShapeModule([trans1], 1)
#mod = multimodule.background
#gd = mod.manifold.gd

#mod = modules.background
#gd = mod.manifold.gd

test_gradcheck_background()


In [ ]:
def test_gradcheck_cot_to_vs():
    def background_gd(gd):  
        
        man = mod.manifold.copy()
        man[0].fill_cotan(torch.ones(mod.controls[0].shape).view(-1,1)) 
        man[0].fill_cotan(mod.controls[0].view(-1,1), copy=True)

        field = man.cot_to_vs(1)
        return field(points)
    
    return torch.autograd.gradcheck(background_gd, (gd), raise_exception=True)

mod = modules.background
#mod = modules.module_list[0]

modules.compute_geodesic_control(modules.manifold)
#mod.fill_controls_zero

gd = mod.manifold.gd
points = torch.rand(dim*nb_pts1).view(-1,2)


test_gradcheck_cot_to_vs()

In [ ]:
def test_gradcheck_background_gd():
    def background_gd(gd):
        #field = mod.field_generator()    
        
        man = mod.manifold.copy()
        for i in range(len(mod.module_list)):
            man[i].fill_cotan(mod.controls[i].view(-1))
        field = man.cot_to_vs(1)
        
        return field(points)
    
    return torch.autograd.gradcheck(background_gd, (gd), raise_exception=True)

mod = modules.background
#mod = modules.module_list[0]

gd = mod.manifold.gd
#gd = [torch.ones(mod.manifold.gd[0].shape, requires_grad=True)]

points = torch.rand(dim*nb_pts1).view(-1,2)

test_gradcheck_background_gd()

In [ ]:
def test_gradcheck_background_field():
    def background_field(gd):
            
        return mod.field_generator()(gd.view(-1,dim))
    
    return torch.autograd.gradcheck(background_field, (gd), raise_exception=True)

mod = modules.background

#gd = [mod.manifold.gd[0]]
gd = mod.manifold.gd
#gd = [torch.ones(mod.manifold.gd[0].shape, requires_grad=True)]

test_gradcheck_background_field()

#print(gd[0].requires_grad, mom[0].requires_grad)

In [ ]:
def test_gradcheck_innerprod():
    def innerprod(gd, mom):
        
        mod.manifold.fill_gd([gd])
        mod.manifold.fill_cotan([mom])   
                
        return mod.manifold.inner_prod_field(mod.field_generator())
    
    return torch.autograd.gradcheck(innerprod, (gd, mom), raise_exception=True)

mod = modules.background

gd = mod.manifold.gd[0]
mom = mod.manifold.cotan[0]

test_gradcheck_innerprod()

In [ ]:
def test_gradcheck_innerprod():
    def innerprod(gd, mom, controls, l):
        
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        #H.module.fill_controls(controlstensor2list(controls, [nb_pts1, nb_pts2], dim))
        H.module.fill_l(l)
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))
        H.module.fill_controls(controlstensor2list(controls, [nb_pts1], dim))
        
        
        return sum([H.module.manifold.manifold_list[i].inner_prod_field(H.module.module_list[i].field_generator()) for i in range(2)])
    
    modules.compute_geodesic_variables(modules.manifold, constr_Id)
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    controls = controlslist2tensor(modules.controls)
    return torch.autograd.gradcheck(innerprod, (gd, mom, controls, modules.l), raise_exception=True)

test_gradcheck_innerprod()

In [ ]:
def test_gradcheck_autoaction():
    def autoaction(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        return H.module.autoaction()


    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(autoaction, (gd, mom), raise_exception=True)


test_gradcheck_autoaction()

In [ ]:
def test_gradcheck_geodesic_cont_self():
    def geodesic_cont_self(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        trans1.manifold.fill_gd(gd)
        trans1.manifold.fill_cotan(mom)

        trans1.compute_geodesic_control_from_self()
        return controlslist2tensor(trans1.controls)

    gd = modules.manifold.gd[0]
    mom = modules.manifold.cotan[0]

    return torch.autograd.gradcheck(geodesic_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_cont_self()

In [ ]:
def test_gradcheck_geo_cont_self():
    def geodesic_geo_cont_self(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
                
        H.module.manifold.manifold_list[j].fill_gd(gd)
        H.module.manifold.manifold_list[j].fill_cotan(mom)
                
        #print(H.module.manifold.gd[j])
        #print(H.module.manifold.cotan[j])

        
        return controlslist2tensor(H.module.controls[1])

    j = 1
    gd = modules.manifold.gd[j]
    mom = modules.manifold.cotan[j]

    return torch.autograd.gradcheck(geodesic_geo_cont_self, (gd, mom), raise_exception=True)


test_gradcheck_geo_cont_self()

In [ ]:
def test_gradcheck_geodesic_controls():
    def geodesic_controls(gd, mom):        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        return controlslist2tensor(H.module.controls)

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    

    return torch.autograd.gradcheck(geodesic_controls, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_controls()

In [ ]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.module.compute_geodesic_variables(modules.manifold, constr_Id)
        return H.module.l

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)

    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


test_gradcheck_geodesic_var()

In [ ]:
def test_gradcheck_geodesic_var():
    def geodesic_var(gd, mom):
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))

        H.module.compute_geodesic_variables(H.module.manifold, constr_Id)
        return H.module.l

    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)


    return torch.autograd.gradcheck(geodesic_var, (gd, mom), raise_exception=True)


#test_gradcheck_geodesic_var()

In [ ]:
def test_gradcheck_apply_mom():
    def apply_mom(gd, mom, controls, l):

        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        #H.module.fill_controls(controlstensor2list(controls, [nb_pts1, nb_pts2], dim))
        
        H.module.fill_l(l)
        
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))
        H.module.fill_controls(controlstensor2list(controls, [nb_pts1], dim))
        
        return H.apply_mom()

    modules.compute_geodesic_variables(modules.manifold, constr_Id)
    gd = gdlist2tensor(modules.manifold.gd)
    mom = controlslist2tensor(modules.manifold.cotan)
    controls = controlslist2tensor(modules.controls)

    return torch.autograd.gradcheck(apply_mom, (gd, mom, controls, modules.l), raise_exception=True)

#test_gradcheck_apply_mom()

In [11]:
def test_gradcheck_Hamiltonian():
    def hamiltonian(gd, mom, l):
        
        H.module.fill_l(l)
        #H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1], dim))
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1], dim))
        #H.module.fill_controls(controlstensor2list(controls, [nb_pts1], dim))
        H.module.manifold.fill_gd(gd)
        H.module.manifold.fill_cotan(mom)
        H.geodesic_controls()
        
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom, modules.l), raise_exception=True)

modules.compute_geodesic_variables(constr_Id)

#gd = gdlist2tensor(modules.manifold.gd)
#mom = controlslist2tensor(modules.manifold.cotan)
#controls = controlslist2tensor(modules.controls)
gd = torch.cat(modules.manifold.unroll_gd())
mom = torch.cat(modules.manifold.unroll_cotan())
#controls = modules.controls
l = modules.l

test_gradcheck_Hamiltonian()

../defmod/deformationmodules.py:95: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  controls, _ = torch.gesv(vs(self.manifold.gd.view(-1, self.manifold.dim)), self.__coeff * K_q)
../defmod/multishape.py:190: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  lambda_qp,_ = torch.gesv(B, A)


torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([1

torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([1

torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([1

torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([1

True

In [13]:
def test_gradcheck_Hamiltonian_2modules():
    def hamiltonian(gd, mom, controls, l):
        
        H.module.fill_l(l)
        H.module.manifold.fill_gd(gdtensor2list(gd, [nb_pts1, nb_pts2], dim))
        H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))
        H.module.fill_controls(controlstensor2list(controls, [nb_pts1, nb_pts2], dim))
        H.geodesic_controls()
        
        return H()
    return torch.autograd.gradcheck(hamiltonian, (gd, mom, controls, modules.l), raise_exception=True)

modules.compute_geodesic_variables(constr_Id)

gd = gdlist2tensor(modules.manifold.gd)
mom = controlslist2tensor(modules.manifold.cotan)
controls = controlslist2tensor(modules.controls)
l = modules.l

test_gradcheck_Hamiltonian_2modules()

torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])


../defmod/deformationmodules.py:95: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  controls, _ = torch.gesv(vs(self.manifold.gd.view(-1, self.manifold.dim)), self.__coeff * K_q)
../defmod/multishape.py:190: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  lambda_qp,_ = torch.gesv(B, A)


TypeError: expected Tensor as element 0 in argument 0, but got list

In [21]:
def grad_H():    
    def hamiltonian(mom):
        
        H.module.manifold.fill_cotan(mom)
        #H.module.manifold.fill_cotan(controlstensor2list(mom, [nb_pts1, nb_pts2], dim))        
        H.geodesic_controls()
        
        return H()
    return torch.autograd.grad(hamiltonian(mom), mom, create_graph=True )

#gd = gdlist2tensor(modules.manifold.gd)
#mom = controlslist2tensor(modules.manifold.cotan)
mom = torch.cat(modules.manifold.unroll_cotan())

grad_H = grad_H()
print(controlstensor2list(grad_H[0], [nb_pts1, nb_pts2], dim))

torch.Size([20, 20]) torch.Size([20, 20])
torch.Size([14, 14]) torch.Size([14, 14])
[tensor([8.1696, 8.1696, 8.1692, 8.1692, 8.1693, 8.1693, 8.1699, 8.1699, 8.1707,
        8.1707, 8.1714, 8.1714, 8.1717, 8.1717, 8.1716, 8.1716, 8.1711, 8.1711,
        8.1703, 8.1703], grad_fn=<SliceBackward>), tensor([-6.0351, -6.0351, -6.0352, -6.0352, -6.0350, -6.0350, -6.0347, -6.0347,
        -6.0344, -6.0344, -6.0345, -6.0345, -6.0348, -6.0348],
       grad_fn=<SliceBackward>), [tensor([2.3726, 2.3726, 2.3722, 2.3722, 2.3723, 2.3723, 2.3729, 2.3729, 2.3737,
        2.3737, 2.3744, 2.3744, 2.3748, 2.3748, 2.3747, 2.3747, 2.3741, 2.3741,
        2.3733, 2.3733], grad_fn=<SliceBackward>), tensor([-3.6717, -3.6717, -3.6718, -3.6718, -3.6716, -3.6716, -3.6713, -3.6713,
        -3.6711, -3.6711, -3.6711, -3.6711, -3.6714, -3.6714],
       grad_fn=<SliceBackward>)]]


../defmod/deformationmodules.py:95: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  controls, _ = torch.gesv(vs(self.manifold.gd.view(-1, self.manifold.dim)), self.__coeff * K_q)
../defmod/multishape.py:190: UserWarning: torch.gesv is deprecated in favour of torch.solve and will be removed in the next release. Please use torch.solve instead.
  lambda_qp,_ = torch.gesv(B, A)


In [14]:
# action 
speed_action = [gdi.action(modulei).tan for gdi, modulei in zip(H.module.manifold.manifold_list, H.module)] 

print(speed_action)
H.module.manifold.fill_tan(speed_action)

sum([H.module.manifold.manifold_list[i].inner_prod_field(H.module.module_list[i].field_generator()) for i in range(3)])

[[tensor([8.8427, 8.8427, 8.8423, 8.8423, 8.8424, 8.8424, 8.8430, 8.8430, 8.8439,
        8.8439, 8.8446, 8.8446, 8.8449, 8.8449, 8.8448, 8.8448, 8.8442, 8.8442,
        8.8434, 8.8434], grad_fn=<ViewBackward>)], [tensor([-6.4336, -6.4336, -6.4337, -6.4337, -6.4335, -6.4335, -6.4332, -6.4332,
        -6.4329, -6.4329, -6.4330, -6.4330, -6.4333, -6.4333],
       grad_fn=<ViewBackward>)], [tensor([2.2287, 2.2287, 2.2286, 2.2286, 2.2287, 2.2287, 2.2288, 2.2288, 2.2289,
        2.2289, 2.2291, 2.2291, 2.2291, 2.2291, 2.2291, 2.2291, 2.2290, 2.2290,
        2.2288, 2.2288], grad_fn=<ViewBackward>), tensor([-3.4846, -3.4846, -3.4846, -3.4846, -3.4846, -3.4846, -3.4846, -3.4846,
        -3.4846, -3.4846, -3.4846, -3.4846, -3.4846, -3.4846],
       grad_fn=<ViewBackward>)]]


tensor(360.3006, grad_fn=<AddBackward0>)